In [33]:
import json
import os
import folium
import pandas as pd
import webbrowser
import matplotlib.pyplot as plt
import base64
import random
import chardet

In [34]:
# ✅ 先检测文件编码
with open("/Users/qihanfei/Desktop/gofodashboard/TESTLAX225.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # 读取部分内容进行检测
    detected_encoding = result["encoding"]
    print(f"✅ 检测到的文件编码：{detected_encoding}")

# ✅ 使用检测到的编码读取 CSV
shipment_df = pd.read_csv("/Users/qihanfei/Desktop/gofodashboard/TESTLAX2_23.csv", encoding=detected_encoding, sep="\t")


✅ 检测到的文件编码：UTF-16


In [43]:
import pandas as pd
import chardet

# ✅ 1. 自动检测 CSV 编码
csv_path = "/Users/qihanfei/Desktop/gofodashboard/TESTLAX225.csv"

with open(csv_path, "rb") as f:
    result = chardet.detect(f.read(100000))  # 读取部分内容进行检测
    detected_encoding = result["encoding"]
    print(f"✅ 检测到的文件编码：{detected_encoding}")

# ✅ 2. 读取 CSV（使用 `encoding_errors="replace"` 处理编码错误）
shipment_df = pd.read_csv(csv_path, encoding=detected_encoding, encoding_errors="replace", sep="\t")

# ✅ 3. 统一列名，确保 `对象` → `zipcode`
shipment_df.rename(columns={"对象": "zipcode"}, inplace=True)

# ✅ 4. 处理 ZIP Code（去除 NaN & "合计"）
shipment_df["zipcode"] = shipment_df["zipcode"].astype(str).str.strip()

# ✅ 5. 提取 **站点合计数据**（`zipcode="合计"` 的行）
site_summary = shipment_df[shipment_df["zipcode"] == "合计"].copy()

# ✅ 6. 确保 `site_summary` 只有一行
site_summary = site_summary.groupby("站点", as_index=False).agg(
    {"领件票数": "sum", "Final妥投量": "sum"}  # **合计每个站点的汇总数据**
)

# ✅ 7. 计算 `Final妥投率`（用合计行计算）
site_summary["Final妥投率"] = site_summary["Final妥投量"] / site_summary["领件票数"]
site_summary.fillna(0, inplace=True)

# ✅ 8. 站点数据映射（**确保唯一索引**）
site_data = site_summary.set_index("站点")[["领件票数", "Final妥投量", "Final妥投率"]].to_dict(orient="index")

# ✅ 9. 处理完成，打印结果
print("✅ 站点数据（合计行）:")
print(site_summary.head())

print("✅ 处理后 `site_data` 结构:")
print(site_data)


✅ 检测到的文件编码：UTF-16
✅ 站点数据（合计行）:
        站点  领件票数  Final妥投量  Final妥投率
0  DEN-CBL   862       828  0.960557
1  DEN-PML   364       356  0.978022
2  DEN-WDL  1138      1130  0.992970
3  DEN-XAE  2614      2424  0.927314
4  DEN-YON   507       498  0.982249
✅ 处理后 `site_data` 结构:
{'DEN-CBL': {'领件票数': 862, 'Final妥投量': 828, 'Final妥投率': 0.9605568445475638}, 'DEN-PML': {'领件票数': 364, 'Final妥投量': 356, 'Final妥投率': 0.978021978021978}, 'DEN-WDL': {'领件票数': 1138, 'Final妥投量': 1130, 'Final妥投率': 0.9929701230228472}, 'DEN-XAE': {'领件票数': 2614, 'Final妥投量': 2424, 'Final妥投率': 0.9273144605967866}, 'DEN-YON': {'领件票数': 507, 'Final妥投量': 498, 'Final妥投率': 0.9822485207100592}, 'LAS-FTN': {'领件票数': 425, 'Final妥投量': 413, 'Final妥投率': 0.971764705882353}, 'LAS-JEE': {'领件票数': 669, 'Final妥投量': 650, 'Final妥投率': 0.9715994020926756}, 'LAS-LLE': {'领件票数': 538, 'Final妥投量': 534, 'Final妥投率': 0.9925650557620818}, 'LAV-CNS': {'领件票数': 332, 'Final妥投量': 326, 'Final妥投率': 0.9819277108433735}, 'LAV-DMIS': {'领件票数': 585, 'Final妥投量': 576, 'Fin

In [44]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager

# ✅ 自动查找可用的中文字体
def get_best_chinese_font():
    fonts = ["Microsoft YaHei", "SimHei", "WenQuanYi Zen Hei", "Arial Unicode MS"]
    for font in fonts:
        if font in [f.name for f in font_manager.fontManager.ttflist]:
            print(f"✅ 使用字体: {font}")
            return font
    print("⚠️ 未找到可用的中文字体，可能会显示乱码！")
    return "sans-serif"  # 兜底方案

# ✅ 设置 Matplotlib 中文字体
best_font = get_best_chinese_font()
matplotlib.rcParams["font.family"] = best_font
matplotlib.rcParams["axes.unicode_minus"] = False  # 处理负号问题

✅ 使用字体: Arial Unicode MS


In [45]:
import matplotlib.pyplot as plt
import base64
import os

# ✅ 确保站点饼图存储路径
pie_chart_paths = {}
output_dir = "site_pie_charts"
os.makedirs(output_dir, exist_ok=True)

# ✅ 生成站点饼图
for _, row in site_summary.iterrows():
    site = row["站点"]
    delivered = row["Final妥投量"]
    not_delivered = row["领件票数"] - delivered  # 计算未妥投量

    if delivered < 0 or not_delivered < 0:  # 避免负值
        continue

    labels = ["Final妥投量", "未妥投量"]
    sizes = [delivered, not_delivered]
    colors = ["green", "red"]

    # 🎨 创建饼图
    plt.figure(figsize=(2, 2))
    plt.pie(sizes, labels=labels, autopct="%1.1f%%", colors=colors, startangle=140)
    plt.axis("equal")

    # 📁 保存为图片
    img_path = os.path.join(output_dir, f"pie_chart_{site}.png")
    plt.savefig(img_path, bbox_inches="tight", dpi=100)
    plt.close()

    # 🔄 转换为 base64 编码的 HTML 图片
    with open(img_path, "rb") as img_file:
        base64_str = base64.b64encode(img_file.read()).decode("utf-8")
        pie_chart_paths[site] = f'<img src="data:image/png;base64,{base64_str}" width="150px"/>'

print("✅ 站点饼图生成完成！")


✅ 站点饼图生成完成！


In [46]:
# ✅ 6. 提取运单涉及的 ZIP Code
shipment_zipcodes = shipment_df["zipcode"].unique().tolist()
print(f"✅ 运单涉及 {len(shipment_zipcodes)} 个唯一 ZIP Code")

# ✅ 7. 加载 ZIP Code GeoJSON 文件
geojson_path = "/Users/qihanfei/Desktop/gofodashboard/merged_zip_codes.geojson"  # 修改为你的文件路径

try:
    with open(geojson_path, "r", encoding="utf-8") as f:
        zipcode_geojson = json.load(f)
    print("✅ ZIP Code 边界文件加载成功！")
except FileNotFoundError:
    print(f"❌ 文件 {geojson_path} 不存在！")
    exit()



✅ 运单涉及 1141 个唯一 ZIP Code
✅ ZIP Code 边界文件加载成功！


In [48]:
import json
import folium
import networkx as nx
import pandas as pd

# ✅ 定义有限颜色集（减少颜色数量但保证相邻不同）
color_palette = ["blue", "purple", "orange", "brown", "pink", "cyan", "gray"]

# ✅ 站点相邻关系（假设 ZIP Code 近似站点相邻）
site_neighbors = {}  # 记录站点邻居

for site in shipment_df["站点"].unique():
    site_neighbors[site] = set(shipment_df[shipment_df["站点"] == site]["zipcode"])

# ✅ 计算站点着色（相邻站点不同色）
G = nx.Graph()
for site, zipcodes in site_neighbors.items():
    G.add_node(site)
    for other_site, other_zipcodes in site_neighbors.items():
        if site != other_site and not zipcodes.isdisjoint(other_zipcodes):
            G.add_edge(site, other_site)

# ✅ 分配颜色
site_color_map = nx.coloring.greedy_color(G, strategy="largest_first")
site_colors = {site: color_palette[color % len(color_palette)] for site, color in site_color_map.items()}

# ✅ **确保 `site_summary` 结构正确**
print("📊 site_summary 数据预览:")
print(site_summary.head())

# ✅ 仅保留运单中涉及的 ZIP Code，并添加统计数据
filtered_features = []

for feature in zipcode_geojson["features"]:
    zip_code = str(feature["properties"].get("ZCTA5CE10", "")).zfill(5)

    if zip_code in shipment_zipcodes:
        site_info = shipment_df[shipment_df["zipcode"] == zip_code].iloc[0]  # 获取站点信息
        site_name = site_info["站点"]

        # **检查站点数据是否存在**
        if site_name not in site_summary["站点"].values:
            print(f"⚠️ 站点 {site_name} 在 site_summary 中不存在，跳过！")
            continue

        site_data = site_summary[site_summary["站点"] == site_name].iloc[0]

        # **修正 `total_tickets` 为 `领件票数`**
        total_tickets = int(site_data["领件票数"])
        total_delivered = int(site_data["Final妥投量"])
        final_rate = float(site_data["Final妥投率"])

        # **三色分类**
        if final_rate < 0.95:
            color = "red"
            border_color = "darkred"
            border_weight = 3.5
            opacity = 0.6
        elif final_rate >= 0.99:
            color = "green"
            border_color = "darkgreen"
            border_weight = 2
            opacity = 0.5
        else:
            color = site_colors.get(site_name, "blue")
            border_color = "black"
            border_weight = 0.5
            opacity = 0.4

        # 更新 GeoJSON 信息
        feature["properties"].update({
            "站点": site_name,
            "总领件票数": total_tickets,
            "总妥投量": total_delivered,
            "合计Final妥投率": f"{final_rate:.2%}",
            "PieChart": pie_chart_paths.get(site_name, ""),
            "fillColor": color,
            "color": border_color,
            "weight": border_weight,
            "fillOpacity": opacity
        })
        filtered_features.append(feature)

filtered_geojson = {
    "type": "FeatureCollection",
    "features": filtered_features
}

print(f"✅ 过滤后，地图上仅显示 {len(filtered_features)} 个 ZIP Code")

# ✅ 创建地图
us_map = folium.Map(location=[39.8283, -98.5795], zoom_start=5, tiles="cartodbpositron")

# ✅ 颜色映射（增加边界加深逻辑）
def zip_code_style(feature):
    """根据站点 Final妥投率 设定颜色"""
    return {
        "fillColor": feature["properties"]["fillColor"],
        "color": feature["properties"]["color"],
        "weight": feature["properties"]["weight"],
        "fillOpacity": feature["properties"]["fillOpacity"]
    }

# ✅ 添加 ZIP Code 区域
folium.GeoJson(
    filtered_geojson,
    name="Filtered ZIP Code Boundaries",
    style_function=zip_code_style,
    tooltip=folium.GeoJsonTooltip(
        fields=["ZCTA5CE10", "站点", "总领件票数", "总妥投量", "合计Final妥投率", "PieChart"],
        aliases=["ZIP Code:", "站点:", "总领件票数:", "总妥投量:", "合计Final妥投率:", "妥投率饼图:"],
        localize=True
    )
).add_to(us_map)

# ✅ 保存地图
output_file = "shipment_site_map.html"
us_map.save(output_file)

print(f"📍 生成地图（按站点颜色标记）：{output_file}")


📊 site_summary 数据预览:
        站点  领件票数  Final妥投量  Final妥投率
0  DEN-CBL   862       828  0.960557
1  DEN-PML   364       356  0.978022
2  DEN-WDL  1138      1130  0.992970
3  DEN-XAE  2614      2424  0.927314
4  DEN-YON   507       498  0.982249
✅ 过滤后，地图上仅显示 1128 个 ZIP Code
📍 生成地图（按站点颜色标记）：shipment_site_map.html


In [49]:
# 提取 Final妥投率 < 95% 的站点，并复制一份用于显示
summary_df = site_summary[site_summary["Final妥投率"] < 0.95].copy()

# 将 Final妥投率转换为百分比格式
summary_df["Final妥投率"] = summary_df["Final妥投率"].apply(lambda x: f"{x:.2%}")

# 按妥投率升序排序
summary_df = summary_df.sort_values(by="Final妥投率", ascending=True)

# 生成 HTML 表格
summary_html = summary_df.to_html(classes="table table-striped", border=0)

summary_file = "summary.html"
with open(summary_file, "w", encoding="utf-8") as f:
    f.write(f"<h2>📢 妥投率 < 95% 的站点（按妥投率排序）</h2>{summary_html}")

button_html = f"""
<div style="
    position: fixed;
    top: 10px; left: 10px;
    z-index: 1000;
    background-color: white;
    padding: 10px;
    border-radius: 5px;
    box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
">
    <a href="{summary_file}" target="_blank" style="font-size:14px; color:red; text-decoration:none;">
        📊 查看妥投率不达标站点（按妥投率排序）
    </a>
</div>
"""

us_map.get_root().html.add_child(folium.Element(button_html))


In [50]:
# ✅保存 & 打开地图
output_file = "shipment_site_map.html"
us_map.save(output_file)
webbrowser.open(f"file://{os.path.abspath(output_file)}")

print(f"📍 生成地图（按站点颜色标记）：{output_file}")
print(f"📄 生成 Summary 报告（妥投率不达标）：{summary_file}")


📍 生成地图（按站点颜色标记）：shipment_site_map.html
📄 生成 Summary 报告（妥投率不达标）：summary.html
